In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit

v = np.array([0.0106, 0.0212, 0.0318, 0.0424, 0.0531, 0.0637, 0.0743, 0.0849, 0.0955, 0.1061])
dp = np.array([436, 905, 1393, 1921, 2097, 2134, 2146, 2151, 2146, 2136])
df = pd.DataFrame({
    'v0': v,
    'delta_P': dp
})
plt.figure(figsize=(6, 6))
plt.scatter(df['v0'], df['delta_P'], color='Tab:blue')
plt.xlabel('Superficial velocity of air $v_0$ (m s$^{-1}$)',fontsize=14)
plt.ylabel('Pressure drop Δ$P$ (Pa)',fontsize=14)
plt.ylim(0, 2500)
plt.xlim(0,0.12)

In [ ]:
def line_through_origin(x, m):
    return m * x

def line_with_intercept(x, m, c):
    return m * x + c

###
# DEFINE REGIONS
v_thresh = 0.045
dp_thresh = 2000
###

region1 = (v < v_thresh) & (dp < dp_thresh)
region2 = (v > v_thresh) & (dp > dp_thresh)

v1, dp1 = v[region1], dp[region1]
v2, dp2 = v[region2], dp[region2]

popt1, pcov1 = curve_fit(line_through_origin, v1, dp1)
m1 = popt1[0]
dm1 = np.sqrt(np.diag(pcov1))[0]
popt2, pcov2 = curve_fit(line_with_intercept, v2, dp2)
m2, c2 = popt2
dm2, dc2 = np.sqrt(np.diag(pcov2))

v_intersect = c2 / (m1 - m2)
dp_intersect = m1 * v_intersect

fstr1 = f'$m$ = {m1:.1f} ± {dm1:.1f}'
fstr2 = f'$m$ = {m2:.1f} ± {dm2:.1f}\n $b$ = {c2:.1f} ± {dc2:.1f}'

v_extended = np.linspace(0, 0.12, 100)
line1 = line_through_origin(v_extended, m1)
line2 = line_with_intercept(v_extended, m2, c2)

def plot_and_save(filename=None):
    plt.figure(figsize=(6, 6))
    plt.scatter(v, dp, color='Tab:blue', label='Data')
    plt.plot(v_extended, line1, color='Tab:orange')
    plt.plot(v_extended, line2, color='Tab:purple')
    plt.scatter(v_intersect, dp_intersect, color='Tab:red', s=80, label=f'Intersection\n(v = {v_intersect:.3f}, ΔP = {dp_intersect:.1f})')
    plt.axvline(x=v_intersect, color='black', linestyle='--', alpha=0.5)

    plt.text(0.03, 800, fstr1, color='Tab:orange', fontsize=12, bbox=dict(facecolor='white', edgecolor='orange', boxstyle='round,pad=0.5'))
    plt.text(0.06, 1800, fstr2, color='Tab:purple', fontsize=12, bbox=dict(facecolor='white', edgecolor='purple', boxstyle='round,pad=0.5'))

    plt.xlabel('Superficial velocity of air $v$ (m/s)', fontsize=14)
    plt.ylabel('Pressure drop Δ$P$ (Pa)', fontsize=14)
    plt.ylim(0, 2500)
    plt.xlim(0, 0.12)

    if filename:
        plt.savefig(filename, dpi=300, bbox_inches='tight')
    else:
        plt.show()

# plot_and_save("plot-for-fluidized-bed-experiment.png")
plot_and_save()

print(f"Intersection point: v = {v_intersect:.3f}, ΔP = {dp_intersect:.1f}")
print(f"Region 1 Slope: m = {m1:.1f} ± {dm1:.1f}")
print(f"Region 2 Slope: m = {m2:.1f} ± {dm2:.1f}, Intercept: b = {c2:.1f} ± {dc2:.1f}")

The intersection point is defined as:

$$
v_{\text {intersect }}=\frac{c_2}{m_1-m_2}
$$

where:
- $c_2$ : intercept of the line in region 2.
- $m_1$ : slope of the line in region 1 .
- $m_2$ : slope of the line in region 2.

The general formula for the uncertainty in a function $f\left(x_1, x_2, \ldots\right)$ is:

$$
\sigma_f^2=\left(\frac{\partial f}{\partial x_1} \sigma_{x_1}\right)^2+\left(\frac{\partial f}{\partial x_2} \sigma_{x_2}\right)^2+\ldots
$$


For $v_{\text {intersect }}=\frac{c_2}{m_1-m_2}$, the partial derivatives are:

$$
\begin{aligned}
\frac{\partial v_{\text {intersect }}}{\partial m_1} & =-\frac{c_2}{\left(m_1-m_2\right)^2} \\
\frac{\partial v_{\text {intersect }}}{\partial m_2} & =\frac{c_2}{\left(m_1-m_2\right)^2} \\
\frac{\partial v_{\text {intersect }}}{\partial c_2} & =\frac{1}{m_1-m_2}
\end{aligned}
$$

Using the error propagation formula:

$$
\sigma_{v_{\text {intersect }}^2}^2=\left(-\frac{c_2}{\left(m_1-m_2\right)^2} \cdot \sigma_{m_1}\right)^2+\left(\frac{c_2}{\left(m_1-m_2\right)^2} \cdot \sigma_{m_2}\right)^2+\left(\frac{1}{m_1-m_2} \cdot \sigma_{c_2}\right)^2
$$

where:
- $\sigma_{m_1}$ : uncertainty in slope $m_1$ (region 1 ).
- $\sigma_{m_2}$ : uncertainty in slope $m_2$ (region 2 ).
- $\sigma_{c_2}:$ uncertainty in intercept $c_2$ (region 2).

In [ ]:
# Uncertainty in intersection point
dm1 = np.sqrt(np.diag(pcov1))[0]  # Uncertainty in m1
dm2, dc2 = np.sqrt(np.diag(pcov2))  # Uncertainties in m2 and c2

# Partial derivatives
partial_m1 = -c2 / (m1 - m2)**2
partial_m2 = c2 / (m1 - m2)**2
partial_c2 = 1 / (m1 - m2)

# Error propagation
sigma_v_intersect = np.sqrt(
    (partial_m1 * dm1)**2 +
    (partial_m2 * dm2)**2 +
    (partial_c2 * dc2)**2
)

v_intersect_with_error = f"{v_intersect:.3f} ± {sigma_v_intersect:.3f}"

print(f"Velocity at the intersection: v = {v_intersect_with_error}")